
# MINERÍA DE DATOS



## Taller: Preprocesamiento

**Profesora:** Elizabeth León Guzmán

**Tema:** Preprocesamiento

**Entrega:** Entregar documento detallado del trabajo realizado y análisis respectivos.

**Equipo 4:** Paula Velosa- Johan Díaz- Fabián López  



## Actividades


In [1]:
import pandas as pd
import sklearn as skl
import numpy as np


### 1. Normalización de datos


In [7]:
X = np.array([-5.0, 23.0, 17.6, 7.23, 1.11])


Dado un conjunto de datos de una dimensión
**X = {-5.0, 23.0, 17.6, 7.23, 1.11}**, normalizar usando:

* a) Min-max normalización en el intervalo \[0,1]


In [8]:
min_max_0_1 = (X-X.min())/(X.max()-X.min())

* b) Min-max normalización en el intervalo \[-1,1]


In [9]:
min_max_m1_1 = 2*(X-X.min())/(X.max()-X.min())-1

* c) Normalización usando desviación estándar


In [10]:
norm_by_std = (X-X.mean())/X.std()

* d) Escala decimal en el intervalo \[-1,1]


In [11]:
dec_scale=X/(X**(np.ceil(np.log10(X.max()))))

* e) Comparar los resultados y discutir ventajas y desventajas


TypeError: scatter() missing 1 required positional argument: 'y'

<Figure size 640x480 with 0 Axes>

---

### 2. Valores perdidos

Dado un conjunto de datos de 4 dimensiones con valores perdidos:

```
I1 I2 I3 I4  
0  1  1  2  
2  1  ?  1  
1  ?  ?  0  
?  2  1  ?  
2  2  1  0
```

* a) Dado que el dominio para todos los atributos es \[0,1,2], ¿cuál debe ser el número de ejemplos “artificiales” si los valores perdidos son interpretados como “no importa el valor” y ellos son reemplazados con todos los posibles valores para su dominio?
* b) ¿Qué otro método utilizaría para reemplazar los valores perdidos?



---

### 3. Outliers en número de hijos

El número de hijos de diferentes pacientes es dado por el siguiente vector:
**C = {3, 1, 0, 2, 7, 3, 6, 4, -2, 0, 0, 10, 15, 6}**


In [14]:
C = np.array([3, 1, 0, 2, 7, 3, 6, 4, -2, 0, 0, 10, 15, 6])


* a) Encontrar outliers usando parámetros estadísticos estándar: media y varianza


In [16]:
C_normalized = (C-C.mean())/C.std()
C_normalized

array([-0.21030754, -0.66327761, -0.88976265, -0.43679257,  0.69563262,
       -0.21030754,  0.46914758,  0.0161775 , -1.34273272, -0.88976265,
       -0.88976265,  1.37508773,  2.50751292,  0.46914758])

In [ ]:
C[abs(C_normalized)>3]

array([], dtype=int64)

* b) Si el umbral cambia de ±3 desviaciones estándar a ±2 desviaciones estándar, ¿cuál “outlier adicional” se encuentra?


In [18]:
C[abs(C_normalized)>2]

array([15])


---

### 4. Detección de outliers multivariados

Dado un conjunto de tres dimensiones:

```
X = [{1,2,0},{3,1,4},{2,1,5},{0,1,6},{2,4,3},{4,4,2},{5,2,1},{7,7,7},{0,0,0},{3,3,3}]
```


In [20]:
X = np.array([[1,2,0],[3,1,4],[2,1,5],[0,1,6],[2,4,3],[4,4,2],[5,2,1],[7,7,7],[0,0,0],[3,3,3]])
X

array([[1, 2, 0],
       [3, 1, 4],
       [2, 1, 5],
       [0, 1, 6],
       [2, 4, 3],
       [4, 4, 2],
       [5, 2, 1],
       [7, 7, 7],
       [0, 0, 0],
       [3, 3, 3]])


* a) Describir el procedimiento e interpretar los resultados de detección de outliers basado en la media y varianza

    Se propone realizar PCA de un único componente tal que sobre este se puedan aplicar una técninca unidimensional de detección de outliers.


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
x_pca = pca.fit_transform(X)

x_pca


array([[-3.01860763],
       [-0.19030271],
       [-0.24505936],
       [-0.903921  ],
       [ 0.38813939],
       [ 1.04700104],
       [-0.05283928],
       [ 7.33795324],
       [-4.7773096 ],
       [ 0.41494591]])

In [23]:
x_pca_normalized = (x_pca-x_pca.mean())/x_pca.std()
x_pca_normalized

array([[-1.01690579],
       [-0.064109  ],
       [-0.08255537],
       [-0.30451209],
       [ 0.13075604],
       [ 0.35271276],
       [-0.01780045],
       [ 2.47200301],
       [-1.60937571],
       [ 0.1397866 ]])

Sobre nuestros nuevos datos unidimensionales realizamos detección de outliers mediante desviación estándar. El criterio será **más de 2 desviaciones estándar**.

In [25]:
x_pca_normalized_br,_ = np.broadcast_arrays(x_pca_normalized,X)

X[(abs(x_pca_normalized_br)>2)]

array([7, 7, 7])


---

### 5. Weka: valores perdidos, normalización y discretización

Usar el conjunto de datos *iris* en Weka:

* a) Eliminar manualmente valores (15%) en sus atributos para simular valores perdidos. Luego aplicar varios métodos disponibles en Weka para reemplazar esos valores. Discutir:

  * Diferencias entre el valor real y el que lo reemplaza
  * Diferencias entre los métodos
* b) Normalizar usando varios métodos
* c) Discretizar usando varios métodos



---

### 6. Reducción por binning

Dado el conjunto de datos:

```
I1  I2   I3  
1   5.9  3.4  
2   2.1  6.2  
1   1.6  2.8  
2   6.8  5.8  
1   3.1  3.1  
1   8.3  4.1  
2   2.4  5.0  
```

Realizar reducción de valores basado en la técnica de **binning** con el mejor corte:

* a) Dimensión I2 usando la **media** como representante de 2 bins
* b) Dimensión I3 usando el **límite más cercano** como representante de 2 bins
  *(Mostrar pasos)*



---

### 7. Ranking de dimensiones

Dado el conjunto de datos con tres dimensiones de entrada y una clase:

```
I1   I2   I3   C  
2.5  1.6  5.9  0  
7.2  4.3  2.1  1  
3.4  5.8  1.6  1  
5.6  3.6  6.8  0  
4.8  7.2  3.1  1  
8.1  4.9  8.3  0  
6.3  4.8  2.4  1  
```

* Realizar **ranking** de las dimensiones comparando medias y varianzas



---

### 8. Selección de características por entropía

Dado el conjunto de datos donde **X1** y **X2** son numéricas, y **X3** y **X4** categóricas:

```
X1  X2  X3  X4  
2.7 3.4 1   A  
3.1 6.2 2   A  
4.5 2.8 1   B  
5.3 5.8 2   B  
6.6 3.1 1   A  
5.0 4.1 2   B  
```

* a) Aplicar el método de **selección de características basado en entropía** para reducir una dimensión (mostrar pasos)
* b) Implementar un programa para realizar el **ranking de dimensiones** usando entropía



---

### 9. Transformación de atributos en *Adult dataset*

Aplicar al conjunto de datos *Adult* del repositorio de Machine Learning:

* a) Convertir todos los atributos numéricos a categóricos utilizando dos estrategias diferentes
* b) Transformar el conjunto de datos para que **todos los atributos sean numéricos**



---

### 10. PCA en conjunto de datos numéricos

Escoger un conjunto de datos del repositorio de Machine Learning que tenga varias dimensiones numéricas y aplicar **PCA**.

* Describir el nuevo conjunto de datos resultante

